In [13]:
def read_input(filename: str) -> tuple[dict[str, dict[int, int]], list[dict[str, str]]]:
    wires: dict[str, dict[int, int]] = {}
    gates = []
    gate_mode = False
    with open(filename) as f:
        for i, line in enumerate(f):
            line = line.strip()
            if line == '':
                gate_mode = True
                continue

            if gate_mode:
                io = line.split('->')
                obj = io[0].strip().split(' ')
                obj = {'type': obj[1], 'left': obj[0], 'right': obj[2], 'output': io[1].strip()}
                gates.append(obj)
            else:
                w_name = line[0:3]
                w_value = bool(int(line[5]))
                wires[w_name] = w_value
    
    return wires, gates

In [14]:
class Node():
    def __init__(self, type: str, name: str, left: 'Node', right: 'Node', graph):
        self.type = type
        self.name = name
        self.value: bool = None
        self.graph = graph
        self.left = left
        self.right = right

    def calculate(self):
        if self.value is not None:
            return self.value

        self.connect()

        if self.type == 'AND':
            return self.and_gate()
        elif self.type == 'OR':
            return self.or_gate()
        elif self.type == 'XOR':
            return self.xor_gate()
        
    def recalculate(self):
        self.value = None
        return self.calculate()

    def and_gate(self):
        self.value = self.left.calculate() & self.right.calculate()
        return self.value
    
    def or_gate(self):
        self.value = self.left.calculate() | self.right.calculate()
        return self.value
    
    def xor_gate(self):
        self.value = self.left.calculate() ^ self.right.calculate()
        return self.value
    
    def connect(self):
        if isinstance(self.left, str) or isinstance(self.right, str):
            self.left = self.graph[self.left]
            self.right = self.graph[self.right]

    def __str__(self):
        # return f'{self.left} {self.type} {self.right} -> {self.name}'
        return f'{self.name}'
    
    def __repr__(self):
        return f'{self.left} {self.type} {self.right} -> {self.name}'

In [15]:
wires, gates = read_input("example.txt")
graph = {}
for gate in gates:
    graph[gate['output']] = Node(gate['type'], gate['output'], gate['left'], gate['right'], graph)

for wire in wires.keys():
    graph[wire] = Node(wire, wire, None, None, graph)
    graph[wire].value = wires[wire]

def and_er(graph: dict[str, Node]):
    z = []
    for key, value in graph.items():
        if key.startswith('z'):
            z.append((key, int(value.calculate())))

    # order by tuple index 0
    z.sort(key=lambda x: x[0])
    z = [x[1] for x in z]
    z

    res = sum(x * 2**i for i, x in enumerate(z))

    x_bin = list(reversed([int(wires[x]) for x in wires.keys() if x.startswith('x')]))
    # print(f"x: {x_bin}")
    x_int = sum(x * 2**i for i, x in (enumerate(reversed(x_bin))))
    # print(f"x: {x_int}")

    y_bin = list(reversed([int(wires[y]) for y in wires.keys() if y.startswith('y')]))
    # print(f"y: {y_bin}")
    y_int = sum(y * 2**i for i, y in (enumerate(reversed(y_bin))))
    # print(f"y: {y_int}")

    # print(f"res: {res}")
    # print(f"z: {z}")

    return (x_int, y_int, res)

x_int, y_int, res = and_er(graph)

# confirm fails
assert x_int & y_int != res

The machine we built is an adder. We demonstrate above that the machine does not work as expected for the given inputs.

Define $N_x, N_y \in \mathbb{N+}$ as the input integers.
The set $S_x$ is the binary representation of $N_x$ and the set $S_y$ is the binary representation of $N_y$.
These sets are the initial wires.

A gate $g_i$ consists of inputs $O_{i1}$ and $O_{i2}$ and output $O_i$.

There is also a set of gates $G_0 \subseteq G$ that are the initial gates, as they have parent edges from the initial wires $S_x$ and $S_y$.
The graph $G$ is a directed acyclic graph with nodes $g_i$ $|$ parent edges $I_{i1}, I_{i2}$ produces the resulting output a single child edge $O_i$

The leaf nodes are the final outputs of the machine, $O_z$ | binary representation of the sum of $N_x + N_y$.

Due to a set of swapped output pairs $O_sw \subseteq O$, the output of the machine is not correct for any inputs $N_x + N_y$

In [16]:
wires, gates = read_input("example2.txt")
graph: dict[str, Node] = {}
for gate in gates:
    graph[gate['output']] = Node(gate['type'], gate['output'], gate['left'], gate['right'], graph)

for wire in wires.keys():
    graph[wire] = Node(wire, wire, None, None, graph)
    graph[wire].value = wires[wire]

swapped_gates: list[tuple[Node, Node]] = [(graph['z05'], graph['z00']), (graph['z02'], graph['z01'])]
for gate in swapped_gates:
    gate[0].left, gate[1].left = gate[1].left, gate[0].left
    gate[0].right, gate[1].right = gate[1].right, gate[0].right

for key, node in graph.items():
    if key.startswith('z'):
        int(node.recalculate())

x_int, y_int, res = and_er(graph)

# confirm passes
assert x_int & y_int == res

For the real problem we have to find and fix the set of 4 pairs of swapped outputs

$O_{1_a}$, $O_{1_b}$

$O_{2_a}$, $O_{2_b}$

$O_{3_a}$, $O_{3_b}$

$O_{4_a}$, $O_{4_b}$

Such that the machine produces the correct output for any inputs $N_x + N_y$

In [17]:
from itertools import permutations, combinations
def generate_permutations(words, num_pairs):
    def is_valid_combination(combo):
        # Flatten and check if all characters are unique
        used = set()
        for pair in combo:
            for char in pair:
                if char in used:
                    return False
                used.add(char)
        return True
    
    # Step 1: Generate only unique pairs from the words list
    pairs = [(w1, w2) for w1 in words for w2 in words if w1 != w2]
    
    # Step 2: Build combinations of pairs and filter invalid ones
    valid_permutations = []
    combs = combinations(pairs, num_pairs)
    for combo in combs:
        if is_valid_combination(combo):
            valid_permutations.append(combo)
            if len(valid_permutations) > 1000:  # Limit to avoid huge memory
                break

    return valid_permutations

In [18]:
wires, gates = read_input("example2.txt")
graph: dict[str, Node] = {}
for gate in gates:
    graph[gate['output']] = Node(gate['type'], gate['output'], gate['left'], gate['right'], graph)

for wire in wires.keys():
    graph[wire] = Node(wire, wire, None, None, graph)
    graph[wire].value = wires[wire]

# initialize graph
for key, value in graph.items():
    if key.startswith('z'):
        value.calculate()

def unparented(node: Node):
    return node.left is None or node.right is None

# iterate through all possible swaps
def attempt_swap_2_pairs():  
    two_pair_permutations = generate_permutations(graph.keys(), 2)
    print(two_pair_permutations)

    for swap in two_pair_permutations:
        pair_1 = swap[0]
        pair_2 = swap[1]
        swapped_gates: list[tuple[Node, Node]] = [
            (graph[pair_1[0]], graph[pair_1[1]]),
            (graph[pair_2[0]], graph[pair_2[1]]),
        ]
        
        if any(unparented(gate[0]) or unparented(gate[1]) for gate in swapped_gates):
            continue

        for gate in swapped_gates:
            gate[0].left, gate[1].left = gate[1].left, gate[0].left
            gate[0].right, gate[1].right = gate[1].right, gate[0].right

        for key, node in graph.items():
            if key.startswith('z'):
                int(node.recalculate())

        x_int, y_int, res = and_er(graph)
        # confirm passes
        if (x_int & y_int == res):
            print(f"Success: {x_int} & {y_int} == {res}")
            return swapped_gates
        else:
            # print(f"failed with swap {swapped_gates}")
            # revert swap
            for gate in swapped_gates:
                gate[0].left, gate[1].left = gate[1].left, gate[0].left
                gate[0].right, gate[1].right = gate[1].right, gate[0].right

       

swapped_gates = attempt_swap_2_pairs()

[(('z05', 'z02'), ('z01', 'z03')), (('z05', 'z02'), ('z01', 'z04')), (('z05', 'z02'), ('z01', 'z00')), (('z05', 'z02'), ('z01', 'x00')), (('z05', 'z02'), ('z01', 'x01')), (('z05', 'z02'), ('z01', 'x02')), (('z05', 'z02'), ('z01', 'x03')), (('z05', 'z02'), ('z01', 'x04')), (('z05', 'z02'), ('z01', 'x05')), (('z05', 'z02'), ('z01', 'y00')), (('z05', 'z02'), ('z01', 'y01')), (('z05', 'z02'), ('z01', 'y02')), (('z05', 'z02'), ('z01', 'y03')), (('z05', 'z02'), ('z01', 'y04')), (('z05', 'z02'), ('z01', 'y05')), (('z05', 'z02'), ('z03', 'z01')), (('z05', 'z02'), ('z03', 'z04')), (('z05', 'z02'), ('z03', 'z00')), (('z05', 'z02'), ('z03', 'x00')), (('z05', 'z02'), ('z03', 'x01')), (('z05', 'z02'), ('z03', 'x02')), (('z05', 'z02'), ('z03', 'x03')), (('z05', 'z02'), ('z03', 'x04')), (('z05', 'z02'), ('z03', 'x05')), (('z05', 'z02'), ('z03', 'y00')), (('z05', 'z02'), ('z03', 'y01')), (('z05', 'z02'), ('z03', 'y02')), (('z05', 'z02'), ('z03', 'y03')), (('z05', 'z02'), ('z03', 'y04')), (('z05', 'z02

In [19]:
named_gates = [(t[0].name, t[1].name) for t in swapped_gates]

# flatten tuples into single list
named_gates = list([item for sublist in named_gates for item in sublist])
print(named_gates)

['z05', 'z00', 'z02', 'z01']


In [ ]:
wires, gates = read_input("input.txt")
graph: dict[str, Node] = {}
for gate in gates:
    graph[gate['output']] = Node(gate['type'], gate['output'], gate['left'], gate['right'], graph)

for wire in wires.keys():
    graph[wire] = Node(wire, wire, None, None, graph)
    graph[wire].value = wires[wire]

# initialize graph
for key, value in graph.items():
    if key.startswith('z'):
        value.calculate()

def unparented(node: Node):
    return node.left is None or node.right is None

# iterate through all possible swaps
def attempt_swap_4_pairs():  
    four_pair_permutations = generate_permutations(graph.keys(), 4)
    print(four_pair_permutations)

    for swap in four_pair_permutations:
        pair_1 = swap[0]
        pair_2 = swap[1]
        pair_3 = swap[2]
        pair_4 = swap[3]
        swapped_gates: list[tuple[Node, Node]] = [
            (graph[pair_1[0]], graph[pair_1[1]]),
            (graph[pair_2[0]], graph[pair_2[1]]),
            (graph[pair_3[0]], graph[pair_3[1]]),
            (graph[pair_4[0]], graph[pair_4[1]]),        
        ]
        
        if any(unparented(gate[0]) or unparented(gate[1]) for gate in swapped_gates):
            continue

        for gate in swapped_gates:
            gate[0].left, gate[1].left = gate[1].left, gate[0].left
            gate[0].right, gate[1].right = gate[1].right, gate[0].right

        for key, node in graph.items():
            if key.startswith('z'):
                int(node.recalculate())

        x_int, y_int, res = and_er(graph)

        if (x_int + y_int == res):
            print(f"Success: {x_int} + {y_int} == {res}")
            return swapped_gates
        else:
            # revert swap
            for gate in swapped_gates:
                gate[0].left, gate[1].left = gate[1].left, gate[0].left
                gate[0].right, gate[1].right = gate[1].right, gate[0].right

       

swapped_gates = attempt_swap_4_pairs()

In [12]:
named_gates = [(t[0].name, t[1].name) for t in swapped_gates]

# flatten tuples into single list
named_gates = list([item for sublist in named_gates for item in sublist])
print(named_gates)

None


TypeError: 'NoneType' object is not iterable